In [2]:
import pandas as pd
from io import StringIO
import requests
import lxml
import importlib
import json

import importlib
import cache
importlib.reload(cache)

# from helpers import print_percent_done

<module 'cache' from '/Users/oneoPk/code/flavo/flavoenzymes-repo/archive/cache.py'>

# This file is WIP still:
### TODO
        

### QUESTIONS:
- 

### Configurables
Edit the folling options before running the scraper
1. Add the terms to search for on Brenda's website here:
1. Make sure the output and input files are what you want
1. Update the blacklist if new errors arise

In [4]:
useCaching = True # setting this parameter to False will fetch new data from server, instead of using cache
blacklist = [
    # ec number for which requests should not be done (Brenda has some corrupt data)
    # if you run the script and find some ECs produce errors, add them to this list
    '1.1.1.205',
    '1.1.99.15',
    '1.13.11.32',
    '1.14.19.7',
    '1.14.99.40',
    '1.14.99.7',
    '1.3.3.6',
    '1.3.99.B12',
    '1.4.99.B4',
    '1.5.1.29',
    '1.6.8.1',
    '1.6.8.2',
    '1.7.99.5',
    '1.8.1.B1',
    '1.8.99.B1',
    '2.5.1.77',
    '2.7.7.95',
    '3.4.16.4',
    '3.4.21.1',
    '3.4.21.109',
    '3.4.22.3',
    '3.4.22.56',
    '3.4.22.61',
    '3.4.24.15',
    '3.4.24.59',
    '3.4.24.69',
    '7.2.1.1'
]
terms = ['FAD','FMN','flavoenzyme','flavin', 'flavoprotein']
import_file = "export/brenda.json"
export_file = "export/brenda_new_export.json" # made the filename different so that old one can not be overriden

In [3]:
def brenda_request(url):
    headers = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    response = cache.cached_reqest(url, headers=headers, useCaching=useCaching)
    return response
    

In [4]:
def search_ligands_brenda(term):
    columns = ['Ligand','EC Number', 'Role', 'Id', 'Structure', 'Discard']
    url = f'https://www.brenda-enzymes.org/result_download.php?a=13&RN=&RNV=1&os=1&pt=&FNV=1&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&V[1]=1&V[2]=2&W[3]={term}&T[3]=2&nolimit=1'
    response = brenda_request(url)
    df = pd.read_csv(StringIO(response), sep='\t', names=columns)
    return df

In [5]:
def search_enzymes_brenda(term):
    columns = ['EC Number', 'Recommended Name', 'Synonyms', 'Commentary', 'Discard']
    url = f'https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]={term}&T[2]=2&nolimit=1'    
    response = brenda_request(url)
    df = pd.read_csv(StringIO(response), sep='\t', names=columns)
    return df

In [6]:
def brenda_get_enzyme_data(id):
    link = f'https://www.brenda-enzymes.info/enzyme.php?ecno={id}#NATURAL%20SUBSTRATE'
    response = brenda_request(link)
    return response

In [7]:
def search_all_terms(terms, search_fn):
    all_dfs = []

    for term in terms:
        df = search_fn(term)
        if (len(df) < 2):
            print(f'[!] skipping search for {term} since nothing was found')
            continue
        else:
            all_dfs.append(df)
    return pd.concat(all_dfs)

In [8]:
def get_all_ecs(terms):
    
    enzymes_list = set(search_all_terms(terms,search_enzymes_brenda)['EC Number'])
    ligands_list = set(search_all_terms(terms,search_ligands_brenda)['EC Number'])

    ec_set = enzymes_list | ligands_list
    print(f'total ecs found: {len(ec_set)}')

    return ec_set

# SOAP helper 

In [9]:
import zeep


from zeep import Client
import hashlib

email = 'si485@dispostable.com'
password = 'si485@dispostable'

wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256(password.encode("utf-8")).hexdigest()
client = Client(wsdl)

In [10]:
poster_child = '1.14.13.2'

In [11]:
def brendaSOAP(parameters, fn_name):
    fn = client.service[fn_name]
    client.settings.strict = True
    return cache.brenda_cached_reqest(request_name=f'brenda_{fn_name}', params=parameters, request_fn=fn, useCaching=useCaching)

def getSynonyms(ecNumber):
    parameters = (email, password, f"ecNumber*{ecNumber}", 'organism*', 'synonyms*', 'commentary*', 'literature*')
    resp = brendaSOAP(parameters, 'getSynonyms')
    return sorted({synonym['synonyms'] for synonym in resp})

def getReactions(ecNumber):
    parameters = (email, password,f"ecNumber*{ecNumber}", "reaction*", "commentary*", "literature*", "organism*")
    resp = brendaSOAP(parameters, 'getReaction')
    return sorted({reaction['reaction'] for reaction in resp})

def getSystematicName(ecNumber):
    parameters = (email, password,f"ecNumber*{ecNumber}", "organism*", "systematicName*")
    resp = brendaSOAP(parameters, 'getSystematicName')
    return resp[0]['systematicName'] # there should be only one sysname per entry

def getSubstrate(ecNumber):
    parameters = (email, password,f"ecNumber*{ecNumber}", "organism*", "substrate*", "reactionPartners*", "ligandStructureId*")
    resp = brendaSOAP(parameters,'getSubstrate')
    return sorted({substrate['substrate'] for substrate in resp})

def getProduct(ecNumber):
    parameters = (email, password,f"ecNumber*{ecNumber}", "organism*", "product*", "reactionPartners*", "ligandStructureId*")
    resp = brendaSOAP(parameters,'getProduct')
    return sorted({product['product'] for product in resp})

def getPdb(ecNumber):
    parameters = (email, password,f"ecNumber*{ecNumber}", "organism*", "pdb*")
    resp =  brendaSOAP(parameters,'getPdb')
    return sorted({pdb['pdb'] for pdb in resp})

In [12]:
all_ecs = get_all_ecs(terms)

Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]=FAD&T[2]=2&nolimit=1
Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]=FMN&T[2]=2&nolimit=1
Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]=flavoenzyme&T[2]=2&nolimit=1
[!] skipping search for flavoenzyme since nothing was found
Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]=flavin&T[2]=2&nolimit=1
Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=9&RN=&RNV=1&os=1&pt=&FNV=&tt=&SYN=&Textmining=&T[0]=2&T[1]=2&W[2]=flavoprotein&T[2]=2&nolimit=1
Getting cached data...
https://www.brenda-enzymes.org/result_download.php?a=13&RN=&RNV=1&os=1&pt=&FNV=1&t

### Initializing BrendaDB

In [5]:
def read_past_data(path=import_file):
    try:
        with open(path) as json_file:
            return json.load(json_file)
    except:
        return {}

In [14]:
def create_brenda_ec_entry(ec):
    return {
        'SYSNAME': getSystematicName(ec),
        'REACTIONS': getReactions(ec),
        'SYNONYMS': getSynonyms(ec),
        'SUBSTRATE': getSubstrate(ec),
        'PRODUCT': getProduct(ec),
        'PDB': getPdb(ec),
        'EC_NUMBER': ec
    }

In [15]:
def create_brenda_db(list_of_ecs, previous_db, verbose=False):
    new_db = previous_db
    for index, ec in enumerate(list_of_ecs):
        if ec not in previous_db and ec not in blacklist:
            if verbose:
                print(f'Fetching data for {ec} ...')
            try:
                entry = create_brenda_ec_entry(ec)
                sysname = entry['SYSNAME']
                entry_name = ec
                new_db[entry_name] = (entry)
                if verbose: 
                    print_percent_done(index=index, length=len(list_of_ecs))
            except Exception as e:
                print(f"[!] couldn't fetch info for {ec}")
                print("     Error: ", e)
                print("\n\n")
                
    # Writing out the results to the file
    with open(export_file, 'w') as outfile:
        json.dump(new_db, outfile)
    print(f'\nSuccessfully written out {len(new_db)} results to "{export_file}"')

    return new_db

### Running the Update/Scrape Script

In [45]:
# Example of the BrendaDB:
# brenda_enzymes = {
#             ecNumber: {
#                 'SYSNAME': "...", # should this be an array?
#                 'REACTIONS': [{reaction},],
#                 'SYNONYMS': ['...',],
#                 'SUBSTRATE': [{substrate},],
#                 'PRODUCT': [{product},],
#                 'PDB': [{pdb}]
#                 'EC_NUMBER': 'ec:1.1.1.1'
#             },
#         }

previous_db = read_past_data(import_file)
sorted_ecs = sorted(list(all_ecs))
db = create_brenda_db(sorted_ecs, previous_db, verbose=True)

Fetching data for 1.1.1.1 ...


----> Progress: ░░░░░░░░░░ 	0% done
Fetching data for 1.1.1.125 ...


----> Progress: ░░░░░░░░░░ 	0% done
Fetching data for 1.1.1.181 ...


----> Progress: ░░░░░░░░░░ 	0% done
Fetching data for 1.1.1.184 ...


----> Progress: ░░░░░░░░░░ 	0% done
Fetching data for 1.1.1.189 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.193 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.215 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.216 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.217 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.229 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.25 ...


----> Progress: ░░░░░░░░░ 	1% done
Fetching data for 1.1.1.267 ...


----> Progress: ░░░░░░░░░ 	2% done
Fetching data for 1.1.1.27 ...


----> Progress: ░░░░░░░░░ 	2% done
Fetching data for 1.1.1.28 ...


----> Progress: ░░░░░░░░░ 	2% done
Fetching data for 1.1.1.284 ...


-

# Exploration:

In [6]:
db = read_past_data()

In [7]:
len(db)

781

In [50]:
db['2.5.1.9']

{'SYSNAME': '6,7-dimethyl-8-(1-D-ribityl)lumazine:6,7-dimethyl-8-(1-D-ribityl)lumazine 2,3-butanediyltransferase',
 'REACTIONS': ['2 6,7-dimethyl-8-(1-D-ribityl)lumazine = riboflavin + 4-(1-D-ribitylamino)-5-amino-2,6-dihydroxypyrimidine'],
 'SYNONYMS': ['RibD',
  'heavy riboflavin synthase',
  'light riboflavin synthase',
  'lumazine synthase/riboflavin synthase complex',
  'riboflavin synthase',
  'riboflavin synthetase',
  'riboflavine synthase',
  'riboflavine synthetase',
  'synthase, riboflavin'],
 'SUBSTRATE': ['2 6,7-dimethyl-8-ribityllumazine',
  '6,7-dimethyl-8-(1-D-ribityl)lumazine',
  "6,7-dimethyl-8-[1'-(5'-deoxy-D-ribityl)]lumazine",
  '6,7-dimethyl-8-ribityllumazine',
  'more'],
 'PRODUCT': ["4-(1'-D-ribitylamino)-5-amino-2,6-dihydroxypyrimidine",
  '4-(1-D-ribitylamino)-5-amino-2,6-dihydroxypyrimidine',
  "5'-deoxyriboflavin",
  '5-amino-6-ribitylamino-2,4(1H,3H)-pyrimidine-dione',
  '5-amino-6-ribitylamino-2,4-(1H,3H)-pyrimidinedione',
  '?',
  'a compound related to 4

In [39]:
# dataframe to easily work with
df = pd.DataFrame(db).T

#### Discuss: Enzymes that have the same sysname

In [43]:
list_of_sysnames = [previous_db[ec]['SYSNAME'] for ec in sorted(previous_db.keys())]
duplicates = [item for item in list_of_sysnames if list_of_sysnames.count(item) > 1]
[print(ec + " \t|\t " + previous_db[ec]['SYSNAME']) for ec in previous_db.keys() if previous_db[ec]['SYSNAME'] in duplicates]
pass

1.14.13.B1 	|	 (-)-bornane-2,5-dione,FMNH2:oxygen oxidoreductase (1,2-lactonizing)
1.14.14.155 	|	 (-)-bornane-2,5-dione,FMNH2:oxygen oxidoreductase (1,2-lactonizing)
1.3.5.1 	|	 succinate:quinone oxidoreductase
1.3.5.4 	|	 succinate:quinone oxidoreductase
1.6.5.9 	|	 NADH:ubiquinone oxidoreductase
4.2.1.167 	|	 (R)-2-hydroxyglutaryl-CoA hydro-lyase ((E)-glutaconyl-CoA-forming)
4.2.1.B26 	|	 (R)-2-hydroxyglutaryl-CoA hydro-lyase ((E)-glutaconyl-CoA-forming)
7.1.1.2 	|	 NADH:ubiquinone oxidoreductase


In [14]:
df = pd.DataFrame(db).T

In [50]:
df.SUBSTRATE.values.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [28]:
# list = [s for s in df.SUBSTRATE.values]

In [51]:
df

,SYSNAME,REACTIONS,SYNONYMS,SUBSTRATE,PRODUCT,PDB,EC_NUMBER
1.1.1.1,alcohol:NAD+ oxidoreductase,[a primary alcohol + NAD+ = an aldehyde + NADH...,"[(R)-specific alcohol dehydrogenase, 40 kDa al...","[(-)-carvone, (1S,3S)-3-methylcyclohexanol, (2...","[(1R)-2,2,2-trifluoro-1-phenylethanol, (1R)-2,...","[1a4u, 1a71, 1a72, 1adb, 1adc, 1adf, 1adg, 1ag...",1.1.1.1
1.1.1.125,2-deoxy-D-gluconate:NAD+ 3-oxidoreductase,[2-deoxy-D-gluconate + NAD+ = 3-dehydro-2-deox...,"[2-deoxy-D-gluconate 3-dehydrogenase, 2-deoxyg...","[2-deoxy-D-gluconate, NAD+]","[3-dehydro-2-deoxy-D-gluconate, NADH]","[1x1e, 2ekp, 2ekq, 3uf0]",1.1.1.125
1.1.1.181,"cholest-5-ene-3beta,7alpha-diol:NAD+ 3-oxidore...","[cholest-5-ene-3beta,7alpha-diol + NAD+ = 7alp...",[3beta-hydroxy-DELTA5-C27-steroid oxidoreducta...,"[NAD+, cholest-5-ene-3beta,7alpha-diol]","[7alpha-hydroxycholest-4-en-3-one, NADH]",[],1.1.1.181
1.1.1.184,secondary-alcohol:NADP+ oxidoreductase,[R-CHOH-R' + NADP+ = R-CO-R' + NADPH + H+],[15-hydroxyprostaglandin dehydrogenase [NADP+]...,"[(R)-1-phenyl-1,2-ethanediol, (R)-2-butanol, (...","[(1R)-1-(4-fluorophenyl)ethanol, (1R)-1-(4-met...","[1cyd, 1wma, 2hrb, 2pfg, 2zat, 3bhi, 3bhj, 3bh...",1.1.1.184
1.1.1.189,"(5Z,13E)-(15S)-9alpha,11alpha,15-trihydroxypro...","[(5Z,13E)-(15S)-9alpha,11alpha,15-trihydroxypr...","[20-alpha-HSD, 20-alpha-hydroxysteroid dehydro...","[(5Z,13E)-(15S)-11alpha,15-dihydroxy-9-oxopros...","[(5Z,13E)-(15S)-11alpha,15-dihydroxy-9-oxopros...","[1htj, 1ki1, 1q13, 1q5m, 1xcg, 2dls, 2kgr, 2kh...",1.1.1.189
...,...,...,...,...,...,...,...
7.2.1.2,ferredoxin:NAD+ oxidoreductase (Na+-transporting),[2 reduced ferredoxin [iron-sulfur] cluster + ...,"[Fno, NADH:ferredoxin-oxidoreductase, Na+-tran...","[2 reduced ferredoxin, 2 reduced ferredoxin [i...","[2 oxidized ferredoxin, 2 oxidized ferredoxin ...",[],7.2.1.2
7.2.4.5,(2E)-4-carboxybut-2-enoyl-CoA carboxy-lyase [(...,[(2E)-4-carboxybut-2-enoyl-CoA + Na+[side 1] =...,"[GCD, carboxyltransferase, decarboxylase, glut...","[(2E)-4-carboxybut-2-enoyl-CoA, 4-carboxybut-2...","[(2E)-but-2-enoyl-CoA, ?, CO2, Na+[side 2], bu...","[1pix, 3gf3, 3gf7, 3glm, 3gma]",7.2.4.5
7.4.2.3,ATP phosphohydrolase (mitochondrial protein-im...,[ATP + H2O + mitochondrial-protein[side 1] = A...,"[DnaK, DnaK chaperone, HIKESHI, HSP70 ATPase, ...","[ATP, H2O, dihydrofolate reductase[side 1], he...","[?, ADP, dihydrofolate reductase[side 2], heat...","[5aqf, 5aqg, 5aqh, 5aqi, 5aqj, 5aqk, 5aql, 5aq...",7.4.2.3
7.4.2.5,"ATP phosphohydrolase (ABC-type, peptide-export...",[ATP + H2O + protein[side 1] = ADP + phosphate...,"[ABC transport, ABCB10, ABCB2, ABCB3, ABCB8, A...","[5'-L-phenylalanyl-glycyl-floxuridine/in, ADP,...","[5'-L-phenylalanyl-glycyl-floxuridine/out, ?, ...",[3zdq],7.4.2.5


In [54]:
l1 = df.SUBSTRATE.values
flat_list = [item for sublist in l1 for item in sublist]
pd.DataFrame(flat_list)[0].value_counts().head(15)

more                            630
O2                              336
H+                              271
NADPH                           213
NADH                            203
H2O                             167
NAD+                             89
NAD(P)H                          77
acceptor                         77
NADP+                            72
FAD                              59
2 H+                             58
2,6-dichlorophenolindophenol     56
ATP                              51
ferricyanide                     41
Name: 0, dtype: int64

In [55]:
l2 = df.PRODUCT.values
flat_list = [item for sublist in l2 for item in sublist]
pd.DataFrame(flat_list)[0].value_counts().head(15)

?                                       674
H2O                                     268
NADP+                                   204
NAD+                                    191
H+                                      130
H2O2                                    117
NADH                                     83
NAD(P)+                                  76
reduced acceptor                         74
NADPH                                    69
reduced 2,6-dichlorophenolindophenol     67
2 H2O                                    66
CO2                                      62
NH3                                      46
FADH2                                    41
Name: 0, dtype: int64

In [43]:
kegg = read_past_data('export/kegg.json')
df_kegg = pd.DataFrame(kegg).T

In [49]:
kegg_list = df_kegg.SUBSTRATE.dropna().values
flat_list = [item for sublist in kegg_list for item in sublist]
pd.DataFrame(flat_list)[0].value_counts().head(15)

O2 [CPD:C00007]                                          209
H+ [CPD:C00080];                                         131
O2 [CPD:C00007];                                          98
H2O [CPD:C00001]                                          87
NADH [CPD:C00004];                                        83
NADPH [CPD:C00005];                                       82
NADP+ [CPD:C00006]                                        62
H2O [CPD:C00001];                                         60
NAD+ [CPD:C00003];                                        52
H+ [CPD:C00080]                                           44
acceptor [CPD:C00028]                                     38
NAD+ [CPD:C00003]                                         32
ATP [CPD:C00002];                                         31
[reduced NADPH---hemoprotein reductase] [CPD:C03024];     19
NADP+ [CPD:C00006];                                       19
Name: 0, dtype: int64

In [48]:
kegg_list = df_kegg.PRODUCT.dropna().values
flat_list = [item for sublist in kegg_list for item in sublist]
pd.DataFrame(flat_list)[0].value_counts().head(15)

H2O [CPD:C00001]                 156
H+ [CPD:C00080]                  124
NADH [CPD:C00004];                76
H2O2 [CPD:C00027]                 76
NADP+ [CPD:C00006];               75
NADPH [CPD:C00005];               70
NAD+ [CPD:C00003];                67
H2O [CPD:C00001];                 55
reduced acceptor [CPD:C00030]     48
CO2 [CPD:C00011];                 37
H+ [CPD:C00080];                  24
NH3 [CPD:C00014];                 22
CO2 [CPD:C00011]                  20
formaldehyde [CPD:C00067];        19
AMP [CPD:C00020];                 19
Name: 0, dtype: int64

In [44]:
df_kegg

,ENTRY,NAME,CLASS,SYSNAME,REACTION,ALL_REAC,SUBSTRATE,PRODUCT,COMMENT,HISTORY,REFERENCE,PATHWAY,ORTHOLOGY,GENES,DBLINKS,KEGG_ID,EC_NUMBER
2.5.1.9,EC 2.5.1.9 Enzyme,"[riboflavin synthase;, heavy riboflavin syntha...",Transferases; Transferring alkyl o...,"6,7-dimethyl-8-(1-D-ribityl)lumazine:6,7-dimet...","[2, 6,7-dimethyl-8-(1-D-ribityl)lumazine, =, r...",[R00066],"[6,7-dimethyl-8-(1-D-ribityl)lumazine]","[riboflavin [CPD:C00255];, 4-(1-D-ribitylamino...",[A flavoprotein (riboflavin).],EC 2.5.1.9 created 1972,"[{'REFERENCE': '1 [PMID:13944007]', 'AUTHORS'...","{'ec00740': 'Riboflavin metabolism', 'ec01100'...",{'K00793': 'riboflavin synthase'},"{'ATH': 'AT2G20690', 'ALY': '9320283', 'CRB': ...","{'ExplorEnz - The Enzyme Database': '2.5.1.9',...",2.5.1.9,2.5.1.9
2.1.1.343,EC 2.1.1.343 Enzyme,"[8-amino-8-demethylriboflavin N,N-dimethyltran...",Transferases; Transferring one-car...,S-adenosyl-L-methionine:8-amino-8-demethylribo...,"[2, S-adenosyl-L-methionine, +, 8-amino-8-deme...","[R11778, R11779, R11780]","[S-adenosyl-L-methionine [CPD:C00019];, 8-amin...","[S-adenosyl-L-homocysteine [CPD:C00021];, rose...","[The enzyme, characterized from the soil bacte...",EC 2.1.1.343 created 2017,"[{'REFERENCE': '1 [PMID:21911488]', 'AUTHORS'...",{'ec00997': 'Biosynthesis of various secondary...,"{'K22269': '8-amino-8-demethylriboflavin N,N-d...",{'SDV': 'BN159_8032(rosA)'},{'ExplorEnz - The Enzyme Database': '2.1.1.343...,2.1.1.343,2.1.1.343
1.1.5.4,EC 1.1.5.4 Enzyme,"[malate dehydrogenase (quinone);, FAD-dependen...",Oxidoreductases; Acting on the CH-...,(S)-malate:quinone oxidoreductase,"[(S)-malate, +, a, quinone, =, oxaloacetate, +...","[R00361, >, R00360;, (other), R01257]","[(S)-malate [CPD:C00149];, quinone [CPD:C15602]]","[oxaloacetate [CPD:C00036];, reduced quinone [...",[A flavoprotein (FAD). Vitamin K and several o...,"EC 1.1.5.4 created 1978 as EC 1.1.99.16, trans...","[{'REFERENCE': '1', 'AUTHORS': 'Imai, D. and B...","{'ec00020': 'Citrate cycle (TCA cycle)', 'ec00...",{'K00116': 'malate dehydrogenase (quinone)'},"{'PCAD': '112063215', 'PFA': 'PF3D7_0616800', ...","{'ExplorEnz - The Enzyme Database': '1.1.5.4',...",1.1.5.4,1.1.5.4
3.5.99.1,EC 3.5.99.1 Enzyme,[riboflavinase],Hydrolases; Acting on carbon-nitro...,riboflavin hydrolase,"[riboflavin, +, H2O, =, ribitol, +, lumichrome...",[R01732],"[riboflavin [CPD:C00255];, H2O [CPD:C00001]]","[ribitol [CPD:C00474];, lumichrome [CPD:C01727]]",NaN,EC 3.5.99.1 created 1961,"[{'AUTHORS': 'FOSTER JW, YANAGITA T.', 'TITLE'...",{'ec00740': 'Riboflavin metabolism'},NaN,NaN,{'ExplorEnz - The Enzyme Database': '3.5.99.1'...,3.5.99.1,3.5.99.1
6.2.1.49,EC 6.2.1.49 Enzyme,[long-chain fatty acid adenylyltransferase Fad...,Ligases; Forming carbon-sulfur bon...,long-chain fatty acid:holo-[mycocerosate synth...,"[ATP, +, a, long-chain, fatty, acid, +, holo-[...",NaN,"[ATP [CPD:C00002];, long-chain fatty acid [CPD...","[AMP [CPD:C00020];, diphosphate [CPD:C00013];,...","[The enzyme, found in certain mycobacteria, ac...","EC 6.2.1.49 created 2016 as EC 2.7.7.95, trans...","[{'REFERENCE': '1 [PMID:9098059]', 'AUTHORS':...",NaN,{'K12427': 'long-chain fatty acid adenylyltran...,"{'MTU': 'Rv2941(fadD28)', 'MTV': 'RVBD_1521 RV...",{'ExplorEnz - The Enzyme Database': '6.2.1.49'...,6.2.1.49,6.2.1.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.4.21.1,EC 3.4.21.1 Enzyme,"[chymotrypsin;, chymotrypsins A and B;, alpha-...",Hydrolases; Acting on peptide bond...,NaN,"[Preferential, cleavage:, Tyr!,, Trp!,, Phe!,,...",NaN,NaN,NaN,[Chymotrypsin A is formed from cattle and pig ...,EC 3.4.21.1 created 1961 as EC 3.4.4.5 and EC ...,"[{'REFERENCE': '1', 'AUTHORS': 'Wilcox, P.E.',...",NaN,{'K01310': 'chymotrypsin'},"{'HSA': '1504(CTRB1) 440387(CTRB2)', 'PTR': '7...",{'ExplorEnz - The Enzyme Database': '3.4.21.1'...,3.4.21.1,3.4.21.1
1.14.19.7,EC 1.14.19.7 Obsolete Enzyme,[Transferred to 1.11.1.23],Oxidoreductases; Acting on paired ...,NaN,NaN,NaN,NaN,NaN,[Transferred entry: (S)-2-hydroxyp

In [77]:
all = read_past_data('../export/scraped_flavoenzymes.json')

In [78]:
len(all)

839

In [75]:
new_list = {1,2,3}
white_list = {3,4,9}
prev_list = {1,2,3,4}
black_list = set()

In [76]:
master_list = (new_list | white_list) - (prev_list | black_list)
master_list

{9}

In [79]:
set(all.keys())

{'ec:1.1.1.1',
 'ec:1.1.1.125',
 'ec:1.1.1.145',
 'ec:1.1.1.146',
 'ec:1.1.1.181',
 'ec:1.1.1.184',
 'ec:1.1.1.189',
 'ec:1.1.1.193',
 'ec:1.1.1.195',
 'ec:1.1.1.21',
 'ec:1.1.1.213',
 'ec:1.1.1.215',
 'ec:1.1.1.216',
 'ec:1.1.1.217',
 'ec:1.1.1.219',
 'ec:1.1.1.229',
 'ec:1.1.1.239',
 'ec:1.1.1.25',
 'ec:1.1.1.252',
 'ec:1.1.1.267',
 'ec:1.1.1.27',
 'ec:1.1.1.28',
 'ec:1.1.1.284',
 'ec:1.1.1.289',
 'ec:1.1.1.306',
 'ec:1.1.1.328',
 'ec:1.1.1.35',
 'ec:1.1.1.404',
 'ec:1.1.1.47',
 'ec:1.1.1.51',
 'ec:1.1.1.62',
 'ec:1.1.1.64',
 'ec:1.1.1.94',
 'ec:1.1.2.3',
 'ec:1.1.2.4',
 'ec:1.1.3.10',
 'ec:1.1.3.12',
 'ec:1.1.3.13',
 'ec:1.1.3.15',
 'ec:1.1.3.17',
 'ec:1.1.3.19',
 'ec:1.1.3.2',
 'ec:1.1.3.20',
 'ec:1.1.3.21',
 'ec:1.1.3.23',
 'ec:1.1.3.28',
 'ec:1.1.3.37',
 'ec:1.1.3.38',
 'ec:1.1.3.39',
 'ec:1.1.3.4',
 'ec:1.1.3.41',
 'ec:1.1.3.42',
 'ec:1.1.3.43',
 'ec:1.1.3.44',
 'ec:1.1.3.45',
 'ec:1.1.3.46',
 'ec:1.1.3.47',
 'ec:1.1.3.49',
 'ec:1.1.3.5',
 'ec:1.1.3.6',
 'ec:1.1.3.7',
 'ec:1.1.3

In [64]:
df = pd.DataFrame(all).T

In [65]:
df

,SYSNAME,REACTIONS,SYNONYMS,SUBSTRATE,PRODUCT,PDB,EC_NUMBER,ENTRY,NAME,CLASS,REACTION,ALL_REAC,COMMENT,HISTORY,REFERENCE,PATHWAY,ORTHOLOGY,GENES,DBLINKS,KEGG_ID
6.2.1.2,medium-chain fatty acid:CoA ligase (AMP-forming),[ATP + a medium-chain fatty acid + CoA = AMP +...,"[Acyl-activating enzyme, Butyryl-CoA synthetas...","{'ATP': {'name': 'ATP', 'smiles': 'Nc1ncnc2n(c...","{'AMP': {'name': 'AMP', 'smiles': 'Nc1ncnc2n(c...","[2vze, 2wd9, 3b7w, 3c5e, 3day, 3eq6, 3gpc]",6.2.1.2,EC 6.2.1.2 Enzyme,"[medium-chain acyl-CoA ligase;, fadK (gene nam...",Ligases; Forming carbon-sulfur bon...,"[ATP, +, a, medium-chain, fatty, acid, +, CoA,...","[R00389, >, R01176]",[Acts on fatty acids from C4 to C11 and on the...,"EC 6.2.1.2 created 1961, modified 2011, modifi...","[{'REFERENCE': '1 [PMID:13084616]', 'AUTHORS'...","{'ec00650': 'Butanoate metabolism', 'ec01100':...","{'K01896': 'medium-chain acyl-CoA synthetase',...",{'HSA': '116285(ACSM1) 123876(ACSM2A) 142827(A...,"{'ExplorEnz - The Enzyme Database': '6.2.1.2',...",ec:6.2.1.2
1.1.3.17,choline:oxygen 1-oxidoreductase,"[betaine aldehyde + O2 + H2O = betaine + H2O2,...","[APChO-syn, An_CodA, CHO, COX, ChOx, ChOx prot...","{'choline': {'name': 'choline', 'smiles': 'C[N...","{'betaine': {'name': 'betaine', 'smiles': 'C[N...","[2jbv, 3ljp, 3nne, 4mjw]",1.1.3.17,EC 1.1.3.17 Enzyme,[choline oxidase],Oxidoreductases; Acting on the CH-...,"[choline, +, 2, O2, +, H2O, =, betaine, +, 2, ...","[R01022, R08211, R08212]","[A flavoprotein (FAD). In many bacteria, plant...","EC 1.1.3.17 created 1978, modified 2005, modif...","[{'REFERENCE': '1 [PMID:599154]', 'AUTHORS': ...","{'ec00260': 'Glycine, serine and threonine met...",{'K17755': 'choline oxidase'},"{'PAMI': 'JCM7686_pAMI1p088', 'MMC': 'Mmcs_246...",{'ExplorEnz - The Enzyme Database': '1.1.3.17'...,ec:1.1.3.17
2.1.1.83,"S-adenosyl-L-methionine:5,3',4'-trihydroxy-3,7...","[S-adenosyl-L-methionine + 5,3',4'-trihydroxy-...","[4'-OMT, flavonol 4'-O-methyltransferase, meth...",{'S-adenosyl-L-methionine': {'name': 'S-adenos...,{'S-adenosyl-L-homocysteine': {'name': 'S-aden...,[],2.1.1.83,EC 2.1.1.83 Enzyme,"[3,7-dimethylquercetin 4'-O-methyltransferase;...",Transferases; Transferring one-car...,"[S-adenosyl-L-methionine, +, 5,3',4'-trihydrox...",[R03456],"[3,7-Dimethylquercetagetin can also act as acc...",EC 2.1.1.83 created 1989,"[{'REFERENCE': '1 [PMID:3994393]', 'AUTHORS':...",{'ec00944': 'Flavone and flavonol biosynthesis'},NaN,NaN,{'ExplorEnz - The Enzyme Database': '2.1.1.83'...,ec:2.1.1.83
1.3.1.45,vestitone:NADP+ oxidoreductase,[vestitone + NADP+ = 2'-hydroxyformononetin + ...,"[CaIRL, GbIRL1, GmIFR, IFR, IRL, LOC100786578,...","{'vestitone': {'name': 'vestitone', 'smiles': ...",{'2'-hydroxyformononetin': {'name': '2'-hydrox...,[],1.3.1.45,EC 1.3.1.45 Enzyme,"[2'-hydroxyisoflavone reductase;, NADPH:2'-hyd...",Oxidoreductases; Acting on the CH-...,"[vestitone, +, NADP+, =, 2'-hydroxyformononeti...","[R06562;, (other), R06563, R07747, R07751]","[In the reverse reaction, a 2'-hydroxyisoflavo...",EC 1.3.1.45 created 1990,"[{'AUTHORS': 'Tiemann, K., Hinderer, W. and Ba...","{'ec00943': 'Isoflavonoid biosynthesis', 'ec01...",{'K05281': '2'-hydroxyisoflavone reductase'},"{'PSU': 'Psesu_1429', 'PSUW': 'WQ53_15890', 'L...",{'ExplorEnz - The Enzyme Database': '1.3.1.45'...,ec:1.3.1.45
1.4.3.10,putrescine:oxygen oxidoreductase (deaminating),[putrescine + O2 + H2O = 4-aminobutanal + NH3 ...,"[APUO, PO, PUOX, PUT oxidase, PuO, adenine din...","{'putrescine': {'name': 'putrescine', 'smiles'...","{'4-aminobutanal': {'name': '4-aminobutanal', ...","[2yg3, 2yg4, 2yg5, 2yg6, 2yg7, 3rha]",1.4.3.10,EC 1.4.3.10 Enzyme,[putrescine oxidase],Oxidoreductases; Acting on the CH-...,"[putrescine, +, O2, +, H2O, =, 4-aminobutanal,...",[R01151],[A flavoprotein (FAD). 4-Aminobutanal condense...,EC 1.4.3.10 created 1976,"[{'REFERENCE': '1 [PMID:4341347]', 'AUTHORS':...","{'ec00330': 'Arginine and proline metabolism',...",{'K03343': 'putrescine oxidase'},"{'MDX': 'BTO20_02740', '